Start by getting more data.

Include both snotel and weather station data.

In [2]:
import pandas as pd

crystal_summit_seasonal = pd.read_csv('../content/csv/seasonal_weather_crystal_summit.csv')
crystal_summit_seasonal.head()

,Date PDT,Temp F,RH %,Min mph,Spd mph,Gust mph,Dir deg
0,Apr 24 6:00,23,98,7,18,31,W
1,Apr 24 5:00,23,98,6,19,30,W
2,Apr 24 4:00,23,98,7,16,29,W
3,Apr 24 3:00,23,98,4,12,21,W
4,Apr 24 2:00,25,97,0,2,9,NW


In [3]:
crystal_summit_seasonal.tail()

,Date PDT,Temp F,RH %,Min mph,Spd mph,Gust mph,Dir deg
5642,Sep 1 4:00,57,46,10,17,23,W
5643,Sep 1 3:00,57,43,9,16,23,W
5644,Sep 1 2:00,59,41,11,19,27,W
5645,Sep 1 1:00,59,37,8,17,23,W
5646,Sep 1 0:00,59,36,6,15,23,W


In [5]:
morse_lake_seasonal = pd.read_csv('../content/csv/seasonal_snotel_morse_lake.csv')
morse_lake_seasonal.head()

,Date PDT,Temp F,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
0,Apr 24 6:00,30.0,NaN,NaN,57.0,-0.5,126.0,1.0
1,Apr 24 5:00,30.0,NaN,NaN,57.1,-0.2,126.0,1.0
2,Apr 24 4:00,31.0,NaN,NaN,57.2,0.0,123.0,-2.0
3,Apr 24 3:00,31.0,NaN,NaN,57.2,0.2,122.0,-3.0
4,Apr 24 2:00,31.0,NaN,NaN,57.2,0.3,124.0,-2.0


In [6]:
morse_lake_seasonal.tail()

,Date PDT,Temp F,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
5593,Sep 1 5:00,53.0,86.3,0.0,NaN,NaN,0.0,0.0
5594,Sep 1 4:00,57.0,86.3,0.0,NaN,NaN,0.0,0.0
5595,Sep 1 3:00,62.0,86.3,0.0,NaN,NaN,0.0,0.0
5596,Sep 1 2:00,58.0,86.3,0.0,NaN,NaN,0.0,0.0
5597,Sep 1 1:00,58.0,86.3,0.0,0.0,0.0,0.0,0.0


todo: what happens if I drop columns I don't think are useful? How does that affect the prediciton accuracy? How does that align with my knowledge of snow science?

In [7]:
crystal_summit_seasonal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5647 entries, 0 to 5646
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date PDT  5647 non-null   object
 1   Temp F    5647 non-null   int64 
 2   RH %      5647 non-null   int64 
 3   Min mph   5647 non-null   int64 
 4   Spd mph   5647 non-null   int64 
 5   Gust mph  5647 non-null   int64 
 6   Dir deg   5647 non-null   object
dtypes: int64(5), object(2)
memory usage: 308.9+ KB


In [8]:
morse_lake_seasonal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date PDT   5598 non-null   object 
 1   Temp F     5597 non-null   float64
 2   PcpAc in   5068 non-null   float64
 3   ∆Pcp in    5047 non-null   float64
 4   SWE in     5290 non-null   float64
 5   ∆SWE  in   5266 non-null   float64
 6   SnoHt in   4800 non-null   float64
 7   ∆SnoHt in  4417 non-null   float64
dtypes: float64(7), object(1)
memory usage: 350.0+ KB


Join the dataframes (should have same date)

In [9]:
seasonal_weather = crystal_summit_seasonal.join(morse_lake_seasonal, lsuffix=" NWAC", rsuffix=" SNOTEL")

seasonal_weather = seasonal_weather.drop('Date PDT SNOTEL', axis=1)
seasonal_weather = seasonal_weather.rename(columns={'Date PDT NWAC': 'Date PDT'})

seasonal_weather.head()

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
0,Apr 24 6:00,23,98,7,18,31,W,30.0,NaN,NaN,57.0,-0.5,126.0,1.0
1,Apr 24 5:00,23,98,6,19,30,W,30.0,NaN,NaN,57.1,-0.2,126.0,1.0
2,Apr 24 4:00,23,98,7,16,29,W,31.0,NaN,NaN,57.2,0.0,123.0,-2.0
3,Apr 24 3:00,23,98,4,12,21,W,31.0,NaN,NaN,57.2,0.2,122.0,-3.0
4,Apr 24 2:00,25,97,0,2,9,NW,31.0,NaN,NaN,57.2,0.3,124.0,-2.0


Read the danger ratings

In [11]:
seasonal_danger = pd.read_csv('../content/csv/danger_ratings.csv')
# only select target area
target_area = 'West Slopes South'
seasonal_danger = seasonal_danger[(seasonal_danger == target_area).any(axis=1)]
# get rid of days with no rating
no_rating = 'NO RATING'
seasonal_danger = seasonal_danger[seasonal_danger['Danger Rating'] != no_rating]
seasonal_danger.head()

,Date,Area,Danger Rating
15,Apr 15 2023,West Slopes South,MODERATE
27,Apr 14 2023,West Slopes South,LOW
28,Apr 13 2023,West Slopes South,MODERATE
43,Apr 12 2023,West Slopes South,MODERATE
54,Apr 11 2023,West Slopes South,MODERATE


Wind direction -> number

In [12]:
weather_number = { 
    'N': 0, 'NNE': 1, 'NE': 2, 'ENE': 3,
    'E': 4, 'ESE': 5, 'SE': 6, 'SSE': 7,
    'S': 8, 'SSW': 9, 'SW': 10, 'WSW': 11,
    'W': 12, 'WNW': 13, 'NW': 14, 'NNW': 15
}

seasonal_weather['Dir deg'] = seasonal_weather['Dir deg'].map(weather_number)
seasonal_weather.head()

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in
0,Apr 24 6:00,23,98,7,18,31,12,30.0,NaN,NaN,57.0,-0.5,126.0,1.0
1,Apr 24 5:00,23,98,6,19,30,12,30.0,NaN,NaN,57.1,-0.2,126.0,1.0
2,Apr 24 4:00,23,98,7,16,29,12,31.0,NaN,NaN,57.2,0.0,123.0,-2.0
3,Apr 24 3:00,23,98,4,12,21,12,31.0,NaN,NaN,57.2,0.2,122.0,-3.0
4,Apr 24 2:00,25,97,0,2,9,14,31.0,NaN,NaN,57.2,0.3,124.0,-2.0


Connect aggregated weather data to danger ratings

In [24]:
# todo: what happens if we change this?
# todo: how do we weight closer days higher?
# number of previous days to aggregate
days_back = 3

training_data = pd.DataFrame([], columns=seasonal_weather.columns)
training_data['Danger Rating'] = None

for idx, row in seasonal_danger.iterrows():
    date = row['Date'].split(' ')[0] + ' ' + row['Date'].split(' ')[1] # BAD
    date_start = date + ' 0:00'
    idx = seasonal_weather[(seasonal_weather == date_start).any(axis=1)].index[0]
    days_past = seasonal_weather.iloc[idx+1:idx-1+24*days_back]
    # aggregate the data
    agg_past = days_past.drop('Date PDT', axis=1).mean(numeric_only=True)
    agg_past['Date PDT'] = date
    agg_past['Danger Rating'] = row['Danger Rating']
    # put it into the dataframe
    training_data.loc[-1] = agg_past
    training_data.index = training_data.index+1

# todo: validate to make sure I did this right
training_data.head()

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in,Danger Rating
145,Apr 15,21.357143,93.671429,4.742857,10.285714,16.228571,9.9,27.7,NaN,NaN,53.205714,-0.54,122.128571,-1.457143,MODERATE
144,Apr 14,19.657143,94.971429,4.185714,10.385714,17.7,10.728571,26.485714,NaN,NaN,53.745714,-0.76,123.585714,-0.457143,LOW
143,Apr 13,23.185714,96.542857,4.042857,11.228571,21.042857,9.842857,28.971429,NaN,NaN,54.505714,0.505714,124.042857,-0.157143,MODERATE
142,Apr 12,27.4,98.3,5.8,16.085714,31.342857,10.157143,32.657143,NaN,NaN,54.0,1.367143,124.2,0.171429,MODERATE
141,Apr 11,29.185714,96.971429,8.842857,21.128571,38.328571,9.671429,34.128571,NaN,NaN,52.632857,1.632857,124.028571,-1.285714,MODERATE


In [25]:
danger_number = { 'NO RATING': 0, 'LOW': 1, 'MODERATE': 2, 'CONSIDERABLE': 3, 'HIGH': 4, 'EXTREME': 5 }

training_data['Danger Rating'] = training_data['Danger Rating'].map(danger_number)
training_data.head()

,Date PDT,Temp F NWAC,RH %,Min mph,Spd mph,Gust mph,Dir deg,Temp F SNOTEL,PcpAc in,∆Pcp in,SWE in,∆SWE in,SnoHt in,∆SnoHt in,Danger Rating
145,Apr 15,21.357143,93.671429,4.742857,10.285714,16.228571,9.9,27.7,NaN,NaN,53.205714,-0.54,122.128571,-1.457143,2
144,Apr 14,19.657143,94.971429,4.185714,10.385714,17.7,10.728571,26.485714,NaN,NaN,53.745714,-0.76,123.585714,-0.457143,1
143,Apr 13,23.185714,96.542857,4.042857,11.228571,21.042857,9.842857,28.971429,NaN,NaN,54.505714,0.505714,124.042857,-0.157143,2
142,Apr 12,27.4,98.3,5.8,16.085714,31.342857,10.157143,32.657143,NaN,NaN,54.0,1.367143,124.2,0.171429,2
141,Apr 11,29.185714,96.971429,8.842857,21.128571,38.328571,9.671429,34.128571,NaN,NaN,52.632857,1.632857,124.028571,-1.285714,2


Clean up NaN's

Replacing with 0 for now, but todo: is there a better way?

In [26]:
training_data = training_data.fillna(0)

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

# date isn't a number, and isn't relevant (yet)
try:
    training_data = training_data.drop('Date PDT', axis=1)
except:
    pass

drop = 'Danger Rating'
x = training_data.drop(drop, axis=1)
y = training_data[drop]

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

rf_regressor = RandomForestRegressor(random_state=0)

rf_regressor.fit(x_train, y_train)

y_pred_train = rf_regressor.predict(x_train)
r2_score(y_train, y_pred_train)


0.8551208333333333

In [28]:
y_pred = rf_regressor.predict(x_test)
r2_score(y_test, y_pred)

-0.20022954545454508

Wow...it got even worse. ~yay~